In [4]:
from langchain_groq import ChatGroq

In [5]:
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
llm = ChatGroq(model="moonshotai/kimi-k2-instruct")

In [7]:
llm.invoke("What is the capital of France?")

AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 32, 'total_tokens': 40, 'completion_time': 0.013631701, 'prompt_time': 0.001045352, 'queue_time': 0.296447259, 'total_time': 0.014677053}, 'model_name': 'moonshotai/kimi-k2-instruct', 'system_fingerprint': 'fp_c5bd0a648b', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--cb966ba7-e7fa-4b52-889e-230d70e4b262-0', usage_metadata={'input_tokens': 32, 'output_tokens': 8, 'total_tokens': 40})

In [8]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [9]:
embeddings = GoogleGenerativeAIEmbeddings(model='models/embedding-001')

In [10]:
result=embeddings.embed_query("What is the capital of France?")

In [11]:
len(result)

768

1.  Data Ingestion

In [3]:
from langchain.document_loaders import PyPDFLoader

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [13]:
import os 

In [16]:
file_path = os.path.join(os.getcwd(), "data", "graphmae.pdf")

In [18]:
loader = PyPDFLoader(file_path=file_path)

In [20]:
documents = loader.load()

In [23]:
len(documents)

10

In [24]:
documents[0].page_content[:1000]

'GraphMAE2: A Decoding-Enhanced Masked Self-Supervised\nGraph Learner\nZhenyu Hou\nTsinghua University, China\nhouzy21@mails.tsinghua.edu.cn\nYufei He∗\nBeijing Institute of Technology, China\nyufei.he@bit.edu.cn\nYukuo Cen\nTsinghua University, China\ncyk20@mails.tsinghua.edu.cn\nXiao Liu\nTsinghua University, China\nliuxiao21@mails.tsinghua.edu.cn\nYuxiao Dong†\nTsinghua University, China\nyuxiaod@tsinghua.edu.cn\nEvgeny Kharlamov\nBosch Center for Artificial\nIntelligence, Germany\nevgeny.kharlamov@de.bosch.com\nJie Tang†\nTsinghua University, China\njietang@tsinghua.edu.cn\nABSTRACT\nGraph self-supervised learning (SSL), including contrastive and\ngenerative approaches, offers great potential to address the funda-\nmental challenge of label scarcity in real-world graph data. Among\nboth sets of graph SSL techniques, the masked graph autoencoders\n(e.g., GraphMAE)—one type of generative methods—have recently\nproduced promising results. The idea behind this is to reconstruct\nthe no